In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import mode
import xgboost as xgb

%matplotlib inline

# train

In [196]:
#playing with train
train = pd.read_csv(r"C:\Users\rbakshee\Desktop\ooops\AV\loan prediction 3\train_u6lujuX_CVtuZ9i.csv")
del train["Loan_ID"]


train.apply(lambda x : len(x.unique()))


#fill na with null:
train["Gender"].fillna("Gender", inplace = True)
train["Dependents"].fillna("Dependents", inplace = True)
train["Self_Employed"].fillna("Self_Employed", inplace = True)
train["Credit_History"].fillna("Credit_History", inplace = True)

#filling with mod

train["LoanAmount"].fillna(train["LoanAmount"].mode()[0], inplace = True)
train["Married"].fillna(train["Married"].mode()[0], inplace = True)
train["Loan_Amount_Term"].fillna(train["Loan_Amount_Term"].mode()[0], inplace = True)


train["Loan_per_month"] = (train["LoanAmount"]*1000)/train["Loan_Amount_Term"]

train["Income_per_month"] = (train["ApplicantIncome"] + train["CoapplicantIncome"])/12

train[train["Income_per_month"] > train["Loan_per_month"]]
"""
#train.describe()

#train.dtypes

#train.apply(lambda x : len(x.unique()))

categorical_vars = [i for i in train.columns if train[i].dtype == 'O' ]
#categorical_vars


#check if any column is stll having nan
for i in train.columns:
    print(i, pd.isnull(train[i]).sum().sum())
"""
#for the column having only 2 values we will use map function
train["Education"] = train["Education"].map({"Graduate": 1, "Not Graduate": 0})
train["Loan_Status"] = train["Loan_Status"].map({"Y": 1, "N": 0})
train["Married"] = train["Married"].map({"Yes": 1, "No": 0})

#for the rest we use pd.get_dummies()

train2 = pd.get_dummies(train[["Gender", "Dependents","Self_Employed","Credit_History",\
                                  "Property_Area"]], prefix="is_")

#concat this train2 with main train
train = pd.concat([train, train2], axis=1)


#time to delete the object type columns of which we have already made into numeric
del train["Gender"]
del train["Dependents"]
del train["Self_Employed"]
del train["Credit_History"]
del train["Property_Area"]

#rearranging columns of train according to test columns
train = train[['Married', 'Education', 'ApplicantIncome', 'CoapplicantIncome',
       'LoanAmount', 'Loan_Amount_Term', 'Loan_per_month', 'Income_per_month',
       'is__Female', 'is__Gender', 'is__Male', 'is__0', 'is__1', 'is__2',
       'is__3+', 'is__Dependents', 'is__No', 'is__Self_Employed', 'is__Yes',
       'is__0.0', 'is__1.0', 'is__Credit_History', 'is__Rural',
       'is__Semiurban', 'is__Urban', 'Loan_Status']]



# test

In [242]:
#playing with test
test = pd.read_csv(r"C:\Users\rbakshee\Desktop\ooops\AV\loan prediction 3\test_Y3wMUE5_7gLdaTN.csv")
test1 = test["Loan_ID"]
del test["Loan_ID"]

#fill na with null:

test["Gender"].fillna("Gender", inplace = True)

test["Dependents"].fillna("Dependents", inplace = True)
test["Self_Employed"].fillna("Self_Employed", inplace = True)
test["Credit_History"].fillna("Credit_History", inplace = True)

#filling with mod
test["LoanAmount"].fillna(test["LoanAmount"].mode()[0], inplace = True)
test["Loan_Amount_Term"].fillna(test["Loan_Amount_Term"].mode()[0], inplace = True)
test["Married"].fillna(test["Married"].mode()[0], inplace = True)

test["Loan_per_month"] = (test["LoanAmount"]*1000)/test["Loan_Amount_Term"]

test["Income_per_month"] = (test["ApplicantIncome"] + test["CoapplicantIncome"])/12


#for the column having only 2 values we will use map function
test["Education"] = test["Education"].map({"Graduate": 1, "Not Graduate": 0})
test["Married"] = test["Married"].map({"Yes": 1, "No": 0})
                                       

test2 = pd.get_dummies(test[["Gender", "Dependents","Self_Employed","Credit_History",\
                                  "Property_Area"]], prefix="is_")

test = pd.concat([test, test2], axis=1)

#time to delete the object type columns of which we have already made into numeric
del test["Gender"]
del test["Dependents"]
del test["Self_Employed"]
del test["Credit_History"]
del test["Property_Area"]

# scaling

In [221]:
from sklearn.preprocessing import StandardScaler

In [222]:
scaler = StandardScaler()

In [223]:
train_scaled = scaler.fit_transform(train[[x for x in list(train.columns) if x not in ["Loan_Status"]]] )
test_scaled = scaler.transform(test)

## cross validation

In [239]:
seed = 21

In [229]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import xgboost as xgb
from sklearn.model_selection import cross_val_score

## 1- svm

In [ ]:
#cross validation score for svm with un scaled data
best_svm = 0
best_C = 0
best_gamma = 0

for g in [0.001, 0.01, 0.1, 1, 10, 100]:
    for c in [0.001, 0.01, 0.1, 1, 10, 100]:
        clf_svm = svm.SVC(C = c, gamma=g, random_state=seed)
        
        #using svm estimator
        score_svm = cross_val_score(clf_svm, train[[x for x in list(train.columns) if x not in ["Loan_Status"]]], \
                        train["Loan_Status"], cv = 10, scoring = 'accuracy')
        if score_svm.mean() > best_svm:
            best_svm = score_svm.mean()
            best_C = c
            best_gamma = g
        print(score_svm.mean())
            
print("best score", best_svm)
print("best_C", best_C)
print("best_gamma", best_gamma)


In [ ]:
#cross validation score for svm with scaled data
best_svm = 0
best_C = 0
best_gamma = 0

for g in [0.001, 0.01, 0.1, 1, 10, 100]:
    for c in [0.001, 0.01, 0.1, 1, 10, 100]:
        clf_svm_s = svm.SVC(C = c, gamma=g, random_state=seed)
        
        #using svm estimator
        score_svm_s = cross_val_score(clf_svm_s, train_scaled, train["Loan_Status"], cv = 10, scoring = 'accuracy')
        if score_svm_s.mean() >= best_svm:
            best_svm = score_svm_s.mean()
            best_C = c
            best_gamma = g
        print(score_svm_s.mean())
            
print("best score", best_svm)
print("best_C", best_C)
print("best_gamma", best_gamma)


In [257]:
clf = svm.SVC(C=10, gamma=0.001, random_state=seed)

clf.fit(train_scaled, train["Loan_Status"])
predicted = clf.predict(test_scaled)

sample_submission = pd.DataFrame()

some = pd.Series(predicted)

sample_submission = pd.concat([ test1, some], axis=1)

sample_submission.columns = ["Loan_ID", "Loan_Status"]

sample_submission["Loan_Status"] = sample_submission["Loan_Status"].map({1: "Y", 0: "N"})

sample_submission = sample_submission[:367]

sample_submission.to_csv(r"C:\Users\rbakshee\Desktop\ooops\AV\loan prediction 3\sample_submission.csv", index=None)

## 2 - random forest

In [268]:
# cross validation for rf with un scaled data
best = 0
best_e = 0
best_l = 0
score = 0

for e in [500, 1000, 1500]:
    for l in [ 1,5,10,50,100,200,500]:
        RFclf = RandomForestClassifier(n_estimators=e,max_features=25, oob_score='True', n_jobs=-1,min_samples_leaf=l \
                                       ,random_state=seed)

            #using RF estimator
        score = cross_val_score(RFclf, train[[x for x in list(train.columns) if x not in ["Loan_Status"]]], \
                        train["Loan_Status"], cv = 10, scoring = 'accuracy')
         
        if score.mean() >= best:
                best = score.mean()
                best_e = e
                best_l = l
                
                print(score.mean())
            
print("best score", best)
print("best_e", best_e)
print("best_l", best_l)


0.785167837627
0.798074421025
0.809445745511
best score 0.809445745511
best_e 500
best_l 10


In [ ]:
for i in ["sqrt","log2", None ]:
    for j in ["accuracy", 'roc_auc']:
        RFclf = RandomForestClassifier(n_estimators=500,max_features=i, oob_score='True', n_jobs=-1,min_samples_leaf=10 \
                                       ,random_state=seed)

            #using RF estimator
        score = cross_val_score(RFclf, train[[x for x in list(train.columns) if x not in ["Loan_Status"]]], \
                        train["Loan_Status"], cv = 10, scoring =j )
        print(i)  
        print(score)
        print(score.mean())

In [271]:
RFclf = RandomForestClassifier(n_estimators=500, oob_score='True', n_jobs=-1,min_samples_leaf=10 \
                                       ,random_state=seed)

            #using RF estimator
score = cross_val_score(RFclf, train[[x for x in list(train.columns) if x not in ["Loan_Status"]]], \
                        train["Loan_Status"], cv = 10, scoring ='accuracy' )
score.mean()

0.8094457455113192

In [401]:
# cross validation for rf with scaled data
best = 0
best_e = 0
best_l = 0
best_d = 0
score = 0

for e in [120,130,140,150]:
    for l in [ 7,8,9]:
        for d in [2,3,4]:
            RFclf = RandomForestClassifier(n_estimators=e,max_features='auto', oob_score='True', n_jobs=-1,min_samples_leaf=l \
                                       ,max_depth = d,random_state=seed)

            #using RF estimator
            score = cross_val_score(RFclf, train[[x for x in list(train.columns) if x not in ["Loan_Status"]]],  \
                        train["Loan_Status"], cv = 10, scoring = 'accuracy')
         
            if score.mean() >= best:
                best = score.mean()
                best_e = e
                best_l = l
                best_d = d
                print(score.mean())
            
print("best score", best)
print("best_e", best_e)
print("best_l", best_l)
print("best_d", best_d)

0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
0.809445745511
best score 0.809445745511
best_e 150
best_l 9
best_d 4


In [274]:

RFclf = RandomForestClassifier(n_estimators=1500,max_features=25, random_state=seed, min_samples_leaf=40)
RFclf.fit(train_scaled, train["Loan_Status"])


predicted = RFclf.predict(test_scaled)

sample_submission = pd.DataFrame()

some = pd.Series(predicted)

sample_submission = pd.concat([ test1, some], axis=1)

sample_submission.columns = ["Loan_ID", "Loan_Status"]

sample_submission["Loan_Status"] = sample_submission["Loan_Status"].map({1: "Y", 0: "N"})

sample_submission = sample_submission[:367]

sample_submission.to_csv(r"C:\Users\rbakshee\Desktop\ooops\AV\loan prediction 3\sample_submission.csv", index=None)

## 3 - gradient boost machine

In [275]:
from sklearn.ensemble import GradientBoostingClassifier as GBC

In [278]:
gbm = GBC(learning_rate=0.01, n_estimators=500,min_samples_split=500, min_samples_leaf=50, \
          subsample=0.8 , max_depth=8, random_state=seed)
score = cross_val_score(gbm,train[[x for x in list(train.columns) if x not in ["Loan_Status"]]], \
                        train["Loan_Status"], cv = 10, scoring = 'accuracy')
         
print(score)
print(score.mean())


[ 0.68253968  0.68253968  0.68852459  0.68852459  0.68852459  0.68852459
  0.68852459  0.68852459  0.68852459  0.68852459]
0.687327608639


In [291]:
best = 0
best_esti = 0
best_leaf = 0
best_split = 0
score = 0
best_d = 0
counter = 0

for leaf in [10, 20, 50 ]:
    for esti in [100, 500, 1000]:
        for split in [100, 250, 500]:
            for d in [5,10,15,20]:
                gbm = GBC(learning_rate=0.01, n_estimators=esti,min_samples_split=split, min_samples_leaf=leaf, \
                           subsample=0.8 , max_depth=d, random_state=seed)
                score = cross_val_score(gbm,train[[x for x in list(train.columns) if x not in ["Loan_Status"]]], \
                                        train["Loan_Status"], cv = 10, scoring = 'accuracy')
                counter += 1
                
                if score.mean() >= best:
                    print(counter)
                    best = score.mean()
                    best_leaf = leaf
                    best_esti = esti
                    best_split = split
                    best_d = d
                    print(score.mean())
print("best score", best)
print("best_esti", best_esti)
print("best_leaf", best_leaf)
print("best_split", best_split)
print("best_depth", d)

1
0.806167056987
2
0.806167056987
3
0.806167056987
4
0.806167056987
5
0.809445745511
6
0.809445745511
7
0.809445745511
8
0.809445745511
37
0.809445745511
38
0.809445745511
39
0.809445745511
40
0.809445745511
41
0.809445745511
42
0.809445745511
43
0.809445745511
44
0.809445745511
73
0.809445745511
74
0.809445745511
75
0.809445745511
76
0.809445745511
77
0.809445745511
78
0.809445745511
79
0.809445745511
80
0.809445745511
89
0.809445745511
90
0.809445745511
91
0.809445745511
92
0.809445745511
best score 0.809445745511
best_esti 500
best_leaf 50
best_split 250
best_depth 20


In [ ]:
gbm
best score 0.809445745511
best_esti 100
best_lr 0.01
best_split 100
-----------
best score 0.809445745511
best_esti 500
best_leaf 50
best_split 250
best_depth 20

In [336]:
for i in [1,2,5,10,12,15,18,21,23,25]:
    gbm = GBC(learning_rate=0.001, n_estimators=500,min_samples_split=250, min_samples_leaf=10, \
                       max_features = i,subsample=1 , max_depth=50, random_state=seed)
    score = cross_val_score(gbm,train[[x for x in list(train.columns) if x not in ["Loan_Status"]]], \
                                        train["Loan_Status"], cv = 10, scoring = 'accuracy')
    print(score.mean(), i)

0.687327608639 1
0.687327608639 2
0.687327608639 5
0.778558417903 10
0.804579755399 12
0.809445745511 15
0.809445745511 18
0.809445745511 21
0.809445745511 23
0.809445745511 25


In [338]:

gbm =GBC(learning_rate=0.001, n_estimators=500,min_samples_split=250, min_samples_leaf=10, \
                       max_features = 20,subsample=1 , max_depth=50, random_state=seed)
gbm.fit(train_scaled, train["Loan_Status"])


predicted = gbm.predict(test_scaled)

sample_submission = pd.DataFrame()

some = pd.Series(predicted)

sample_submission = pd.concat([ test1, some], axis=1)

sample_submission.columns = ["Loan_ID", "Loan_Status"]

sample_submission["Loan_Status"] = sample_submission["Loan_Status"].map({1: "Y", 0: "N"})

sample_submission = sample_submission[:367]

sample_submission.to_csv(r"C:\Users\rbakshee\Desktop\ooops\AV\loan prediction 3\sample_submission.csv", index=None)

## 4- xgboost

In [345]:
import xgboost

In [383]:
best = 0
best_esti = 0
score = 0
best_d = 0
for esti in [300,301,302]:
    for d in [2,3,4]:
        xgb = xgboost.XGBClassifier(learning_rate=0.001, n_estimators=esti, max_depth=d,seed = seed) 
        score = cross_val_score(xgb, train[[x for x in list(train.columns) if x not in ["Loan_Status"]]], \
                                        train["Loan_Status"], cv = 10, scoring = 'accuracy')
        if score.mean() >= best:
            best = score.mean()
            best_esti = esti
            best_d = d
            print(score.mean())
print("best score", best)
print("best_esti", best_esti)
print("best_depth", best_d)


0.801301066875
0.811137132449
0.812776476711
0.812776476711
best score 0.812776476711
best_esti 302
best_depth 3


In [ ]:
best score 0.812776476711
best_esti 301
best_depth 3

In [390]:
xgb = xgboost.XGBClassifier(learning_rate=0.001, n_estimators=301, max_depth=3,seed = seed) 
score = cross_val_score(xgb, train_scaled, train["Loan_Status"], cv = 10, scoring = 'accuracy')
score.mean()

0.81277647671090292

In [386]:
xgb = xgboost.XGBClassifier(learning_rate=0.001, n_estimators=301, max_depth=3,seed = seed) 
xgb.fit(train_scaled, train["Loan_Status"])


predicted = xgb.predict(test_scaled)

sample_submission = pd.DataFrame()

some = pd.Series(predicted)

sample_submission = pd.concat([ test1, some], axis=1)

sample_submission.columns = ["Loan_ID", "Loan_Status"]

sample_submission["Loan_Status"] = sample_submission["Loan_Status"].map({1: "Y", 0: "N"})

sample_submission = sample_submission[:367]

sample_submission.to_csv(r"C:\Users\rbakshee\Desktop\ooops\AV\loan prediction 3\sample_submission.csv", index=None)

## trying imputing with mode for all vars

In [435]:
#playing with train
train = pd.read_csv(r"C:\Users\rbakshee\Desktop\ooops\AV\loan prediction 3\train_u6lujuX_CVtuZ9i.csv")
del train["Loan_ID"]


#filling with mod
train["Gender"].fillna(train["Gender"].mode()[0], inplace = True)
train["Dependents"].fillna(train["Dependents"].mode()[0], inplace = True)
train["Self_Employed"].fillna(train["Self_Employed"].mode()[0], inplace = True)
train["Credit_History"].fillna(train["Credit_History"].mode()[0], inplace = True)
train["LoanAmount"].fillna(train["LoanAmount"].mode()[0], inplace = True)
train["Married"].fillna(train["Married"].mode()[0], inplace = True)
train["Loan_Amount_Term"].fillna(train["Loan_Amount_Term"].mode()[0], inplace = True)



train["Loan_per_month"] = (train["LoanAmount"]*1000)/train["Loan_Amount_Term"]

train["Income_per_month"] = (train["ApplicantIncome"] + train["CoapplicantIncome"])/12




#for the column having only 2 values we will use map function
train["Education"] = train["Education"].map({"Graduate": 1, "Not Graduate": 0})
train["Loan_Status"] = train["Loan_Status"].map({"Y": 1, "N": 0})
train["Married"] = train["Married"].map({"Yes": 1, "No": 0})

train["Gender"] = train["Gender"].map({"Male":1, "Female" : 0})

train["Dependents"] = train["Dependents"].map({'0':0, '1':1,'2':2, '3+':3})

train["Self_Employed"] = train["Self_Employed"].map({"No":1, "Yes":0})

train["Property_Area"] = train["Property_Area"].map({"Semiurban": 0, "Urban":1, "Rural":2})

### test

In [455]:
test = pd.read_csv(r"C:\Users\rbakshee\Desktop\ooops\AV\loan prediction 3\test_Y3wMUE5_7gLdaTN.csv")
test1 = test["Loan_ID"]
del test["Loan_ID"]

In [456]:

#filling with mod
test["Gender"].fillna(test["Gender"].mode()[0], inplace = True)
test["Dependents"].fillna(test["Dependents"].mode()[0], inplace = True)
test["Self_Employed"].fillna(test["Self_Employed"].mode()[0], inplace = True)
test["Credit_History"].fillna(test["Credit_History"].mode()[0], inplace = True)
test["LoanAmount"].fillna(test["LoanAmount"].mode()[0], inplace = True)
test["Married"].fillna(test["Married"].mode()[0], inplace = True)
test["Loan_Amount_Term"].fillna(test["Loan_Amount_Term"].mode()[0], inplace = True)



test["Loan_per_month"] = (test["LoanAmount"]*1000)/test["Loan_Amount_Term"]

test["Income_per_month"] = (test["ApplicantIncome"] + test["CoapplicantIncome"])/12




#for the column having only 2 values we will use map function
test["Education"] = test["Education"].map({"Graduate": 1, "Not Graduate": 0})
test["Married"] = test["Married"].map({"Yes": 1, "No": 0})

test["Gender"] = test["Gender"].map({"Male":1, "Female" : 0})

test["Dependents"] = test["Dependents"].map({'0':0, '1':1,'2':2, '3+':3})

test["Self_Employed"] = test["Self_Employed"].map({"No":1, "Yes":0})

test["Property_Area"] = test["Property_Area"].map({"Semiurban": 0, "Urban":1, "Rural":2})

In [457]:
#cross validation score for svm with un scaled data
best_svm = 0
best_C = 0
best_gamma = 0

for g in [0.001, 0.01, 0.1, 1, 10, 100]:
    for c in [0.001, 0.01, 0.1, 1, 10, 100]:
        clf_svm = svm.SVC(C = c, gamma=g, random_state=seed)
        
        #using svm estimator
        score_svm = cross_val_score(clf_svm, train[[x for x in list(train.columns) if x not in ["Loan_Status"]]], \
                        train["Loan_Status"], cv = 10, scoring = 'accuracy')
        if score_svm.mean() > best_svm:
            best_svm = score_svm.mean()
            best_C = c
            best_gamma = g
        print(score_svm.mean())
            
print("best score", best_svm)
print("best_C", best_C)
print("best_gamma", best_gamma)


0.687327608639
0.687327608639
0.687327608639
0.684153005464
0.685792349727
0.685792349727
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.690606297164
0.690606297164
0.690606297164
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.690606297164
0.690606297164
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
0.687327608639
best score 0.690606297164
best_C 1
best_gamma 0.1


In [458]:
train_scaled = scaler.fit_transform(train[[x for x in list(train.columns) if x not in ["Loan_Status"]]] )
test_scaled = scaler.transform(test)

In [461]:
#cross validation score for svm with un scaled data
best_svm = 0
best_C = 0
best_gamma = 0

for g in [0.001, 0.01, 0.1, 1, 10, 100]:
    for c in [0.001, 0.01, 0.1, 1, 10, 100]:
        clf_svm = svm.SVC(C = c, gamma=g, random_state=seed)
        
        #using svm estimator
        score_svm = cross_val_score(clf_svm, train_scaled ,\
                        train["Loan_Status"], cv = 5, scoring = 'accuracy')
        if score_svm.mean() > best_svm:
            best_svm = score_svm.mean()
            best_C = c
            best_gamma = g
        print(score_svm.mean())
            
print("best score", best_svm)
print("best_C", best_C)
print("best_gamma", best_gamma)


0.687308302485
0.687308302485
0.687308302485
0.705050237969
0.809571655209
0.809571655209
0.687308302485
0.687308302485
0.687308302485
0.809545214172
0.806292966684
0.79013749339
0.687308302485
0.687308302485
0.776916975145
0.80142781597
0.754309888948
0.721655208884
0.687308302485
0.687308302485
0.687308302485
0.739423585405
0.693786356425
0.696906398731
0.687308302485
0.687308302485
0.687308302485
0.68561607615
0.675806451613
0.679058699101
0.687308302485
0.687308302485
0.687308302485
0.684029613961
0.677472236912
0.677472236912
best score 0.809571655209
best_C 10
best_gamma 0.001
